In [11]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import os

def selecionar_arquivos():
    arquivos = filedialog.askopenfilenames(title="Selecione os arquivos Excel", filetypes=[("Arquivos Excel", "*.xlsx")])
    if arquivos:
        contador_arquivos.set(f"{len(arquivos)} arquivo(s) importado(s)")
        global arquivos_selecionados
        arquivos_selecionados = arquivos

def selecionar_pasta():
    pasta = filedialog.askdirectory(title="Selecione a pasta contendo os arquivos Excel")
    if pasta:
        arquivos = [os.path.join(pasta, arquivo) for arquivo in os.listdir(pasta) if arquivo.endswith('.xlsx')]
        contador_arquivos.set(f"{len(arquivos)} arquivo(s) importado(s)")
        global arquivos_selecionados

        arquivos_selecionados = arquivos

def selecionar_diretorio():
    diretorio_destino = filedialog.askdirectory(title="Selecione o diretório de destino")
    if diretorio_destino:
        sucesso = exportar_analise(diretorio_destino)
        if sucesso:
            messagebox.showinfo("Exportação Concluída", f'Resultado da análise exportado para "{diretorio_destino}/resultado_analise.xlsx".')
        else:
            messagebox.showwarning("Exportação Falhou", "Você precisa realizar uma análise antes de exportar os resultados.")

def analisar_arquivos():
    if arquivos_selecionados:
        progresso['value'] = 0  # Reseta a barra de progresso
        progresso['maximum'] = len(arquivos_selecionados)  # Define o máximo da barra de progresso
        root.update_idletasks()  # Atualiza a interface para exibir a barra de progresso
        combinar_dataframes(arquivos_selecionados)
        messagebox.showinfo("Sucesso", "Análise concluída com sucesso!\n\nExporte para seu diretório.")

def exportar_analise(diretorio_destino):
    if 'df_final' in globals():
        try:
            df_final.to_excel(f"{diretorio_destino}/resultado_analise.xlsx", index=False)
            return True
        except Exception as e:
            print(e)  # Mostrar erro no console para debug
            return False
    else:
        return False

def extrair_dados(arquivo):
    df = pd.read_excel(arquivo)

    # ITERAÇÕES
    desc_dist_acst = ', '.join([str(val) for val in df.iloc[10, 2:].values.flatten() if pd.notna(val)]) if not df.iloc[10, 2:].isnull().all().all() else ''
    obs_gerais = ', '.join([str(val) for val in df.iloc[35:38, 0:].values.flatten() if pd.notna(val)]) if not df.iloc[35:38, 0:].isnull().all().all() else ''

    data = {
        # DADOS GERAIS
        "ID": df.iloc[4, 1],
        "TIPO": df.iloc[3, 1],
        "LOCAL": df.iloc[3, 4],
        "TRECHO": df.iloc[4, 4],
        "RODOVIA": df.iloc[3, 6],
        "DT_INSP": df.iloc[4, 6].strftime('%d/%m/%Y'),
        
        # CADASTRAMENTO
        "KM_INI": df.iloc[6, 1],
        "KM_FIN": df.iloc[7, 1],
        "SENT": df.iloc[6, 4],
        "CORD_INI": df.iloc[6, 6],
        "CORD_FIN": df.iloc[7, 6],
        
        # DADOS GEOMÉTRICOS DO TERRAPLENO
        "EXT": df.iloc[9, 1],
        "ALT_GEO": df.iloc[9, 4],
        "INCLIN": df.iloc[9, 6],
        "DIST_ACST": df.iloc[10, 1],
        "DESC_DIST_ACST": desc_dist_acst,
        
        # CARACTERÍSTICAS GERAIS
        "TIPO_TRPL": df.iloc[12, 1],
        "TIPO_RELEV": df.iloc[12, 6],
        "VGT": df.iloc[13, 1],
        "DENS_VGT": df.iloc[13, 6],

        # DADOS ESTRUTURA DE CONTENCAO
        "TIPO_CON_1": df.iloc[16, 1],
        "*******1": df.iloc[16, 2],
        "TIPO_CON_2": df.iloc[16, 3],
        "*******2": df.iloc[16, 4],
        "TIPO_CON_3": df.iloc[16, 5],
        "TIPO_CON_4": df.iloc[16, 6],

        "EXT_CON1": df.iloc[17, 1],
        "*******3": df.iloc[17, 2],
        "EXT_CON2": df.iloc[17, 3],
        "*******4": df.iloc[17, 4],
        "EXT_CON3": df.iloc[17, 5],
        "EXT_CON4": df.iloc[17, 6],

        "ALT_CON1": df.iloc[18, 1],
        "*******5": df.iloc[18, 2],
        "ALT_CON2": df.iloc[18, 3],
        "*******6": df.iloc[18, 4],
        "ALT_CON3": df.iloc[18, 5],
        "ALT_CON4": df.iloc[18, 6],

        "ANC_CON1": df.iloc[19, 1],
        "*******7": df.iloc[19, 2],
        "ANC_CON2": df.iloc[19, 3],
        "*******8": df.iloc[19, 4],
        "ANC_CON3": df.iloc[19, 5],
        "ANC_CON4": df.iloc[19, 6],

        "ELMT_CON1": df.iloc[20, 1],
        "*******9": df.iloc[20, 2],
        "ELMT_CON2": df.iloc[20, 3],
        "*******10": df.iloc[20, 4],
        "ELMT_CON3": df.iloc[20, 5],
        "ELMT_CON4": df.iloc[20, 6],

        # DRENAGEM
        "DRN_SUP": df.iloc[22, 1],
        "COND_DRN_SUP": df.iloc[22, 5],
        "DRN_SUB": df.iloc[23, 1],
        "TIPO_DRN": df.iloc[23, 4],
        "COND_DRN_SUB": df.iloc[23, 6],

        # CONDIÇÕES GERAIS DE SATURAÇÃO
        "PRES_AGUA": df.iloc[25, 1],
        
        # TIPOS DE INSTABILIZAÇÃO
        "TIPO_OCRR": df.iloc[27, 1],
        
        # DIAGNÓSTICO
        "CAUSAS_PROV": df.iloc[29, 1],
        "PASS_AMBI": df.iloc[30, 1],
        "DESC_PASS_AMBI": df.iloc[30, 2],
        
        # GRAVIDADE DA SITUAÇÃO
        "NVL_RSC": df.iloc[32, 5],
        "OUTR_ELMNT": df.iloc[33, 5],
        
        # OBSERVAÇÕES GERAIS
        "OBS_GERAIS": obs_gerais,
    }
    return data

def combinar_dataframes(arquivos):
    dados = []
    for i, arquivo in enumerate(arquivos):
        data = extrair_dados(arquivo)
        dados.append(data)
        progresso['value'] = i + 1  # Atualiza o valor da barra de progresso
        root.update_idletasks()  # Atualiza a interface para refletir o progresso
    global df_final
    df_final = pd.DataFrame(dados)

# Criar a janela principal
root = tk.Tk()
root.title("Analisar fichar monitoração")
root.geometry("400x350")

# Rótulo para identificar a seção de importação
lbl_importacao = tk.Label(root, text="Importação de Arquivos", font=("Helvetica", 12, "bold"))
lbl_importacao.pack(pady=(10, 5))

# Botão para selecionar arquivos
btn_selecionar = tk.Button(root, text="Importar arquivo", command=selecionar_arquivos)
btn_selecionar.pack(pady=5)

# Botão para selecionar pasta
btn_selecionar_pasta = tk.Button(root, text="Importar pasta", command=selecionar_pasta)
btn_selecionar_pasta.pack(pady=5)

# Rótulo para identificar a seção de análise
lbl_analise = tk.Label(root, text="Análise de Arquivos", font=("Helvetica", 12, "bold"))
lbl_analise.pack(pady=(10, 5))

# Botão para analisar arquivos
btn_analisar = tk.Button(root, text="Analisar arquivos", command=analisar_arquivos)
btn_analisar.pack(pady=5)

# Barra de progresso
progresso = ttk.Progressbar(root, orient="horizontal", length=300, mode="determinate")
progresso.pack(pady=10)

# Rótulo para identificar a seção de exportação
lbl_exportacao = tk.Label(root, text="Exportação de Resultados", font=("Helvetica", 12, "bold"))
lbl_exportacao.pack(pady=(10, 5))

# Botão para exportar análise
btn_exportar = tk.Button(root, text="Exportar análise", command=selecionar_diretorio)
btn_exportar.pack(pady=5)

# Label para mostrar quantos arquivos foram importados
contador_arquivos = tk.StringVar()
contador_arquivos.set("Para iniciar, importe os arquivos")
lbl_contador = tk.Label(root, textvariable=contador_arquivos)
lbl_contador.pack()

# Executar o loop principal da interface
root.mainloop()